In [4]:
import pandas as pd
import numpy as np
import cv2

In [5]:
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers
from keras import applications
from keras.models import Model

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)


In [6]:
from keras import backend as K

In [7]:
K.set_image_dim_ordering("th")

In [8]:
import os
import math
from keras.utils import np_utils

In [9]:
path = "/home/ubuntu/nbs/courses-master/deeplearning1/data/dogscats/sample"

In [10]:
train_data_dir = path + '/train'
validation_data_dir = path + '/valid'

In [11]:
def fcount(path):    
    listOfclasses = [name for name in os.listdir(path)] 
    return len(listOfclasses),listOfclasses

In [12]:
train_labels,nameOfClasses = fcount(train_data_dir)
validation_labels,nameOfClasses= fcount(validation_data_dir)

In [13]:
print train_labels, validation_labels

2 2


In [14]:
print nameOfClasses

['dogs', 'cats']


In [15]:
def gcd(a, b):
    if(a<b):
        a,b=b,a
    while(b!=0):
        r=b
        b=a%r
        a=r
    return a

In [16]:
datagen = ImageDataGenerator(rescale=1./255.)

In [17]:
batch_size = 1

In [18]:
img_width = img_height = 150

In [19]:
train_generator = datagen.flow_from_directory(train_data_dir,target_size=(img_width,img_height),
                                              batch_size=batch_size,shuffle=False,class_mode=None)

valid_generator = datagen.flow_from_directory(validation_data_dir,target_size=(img_width,img_height),
                                              batch_size=batch_size,shuffle=False,class_mode=None)

Found 14 images belonging to 2 classes.
Found 6 images belonging to 2 classes.


In [20]:
model_vgg = applications.VGG16(include_top=False, weights="imagenet")

In [21]:
model_vgg.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3, None, None)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, None, None)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, None, None)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, None, None)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, None, None)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, None, None)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 128, None, None)   0         
__________

In [24]:
epochs = 1
train_samples = train_labels*7
validation_samples = validation_labels*3
validation_samples

6

In [25]:
bottleneck_features_train = model_vgg.predict_generator(train_generator,train_samples // batch_size)

In [26]:
np.save(open(path+"/bottleneck_features_train.npy","wb"),bottleneck_features_train)

In [33]:
bottleneck_features_validation = model_vgg.predict_generator(valid_generator, validation_samples // batch_size)
np.save(open(path+"/bottleneck_features_validation.npy","wb"), bottleneck_features_validation)

In [50]:
train_data = np.load(open(path+"/bottleneck_features_train.npy","rb"))
b = []
for x in range(len(nameOfClasses)):
    for y in range(1,8):
        b.append(x)
train_labels = np.asarray(b)

In [51]:
valid_data = np.load(open(path+"/bottleneck_features_validation.npy","rb"))
c = []
for x in range(len(nameOfClasses)):
    for y in range(1,4):
        c.append(x)
validation_labels = np.asarray(c)

In [52]:
train_labels = np_utils.to_categorical(train_labels,len(nameOfClasses))
validation_labels = np_utils.to_categorical(validation_labels, len(nameOfClasses))

In [53]:
train_data.shape

(14, 512, 4, 4)

In [54]:
valid_data.shape

(6, 512, 4, 4)

In [55]:
import matplotlib.pyplot as plt
%matplotlib inline

In [56]:
temp = train_data[0]

In [57]:
temp.shape

(512, 4, 4)

In [58]:
model_top = Sequential()
model_top.add(Flatten(input_shape=train_data.shape[1:]))
model_top.add(Dense(256, activation='relu'))
model_top.add(Dropout(0.5))
model_top.add(Dense(len(nameOfClasses), activation='softmax'))
model_top.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [59]:
model_top.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 514       
Total params: 2,097,922
Trainable params: 2,097,922
Non-trainable params: 0
_________________________________________________________________


In [62]:
model_top.fit(train_data, train_labels,
            epochs=30,
            batch_size=batch_size,
            validation_data=(valid_data, validation_labels))

Train on 14 samples, validate on 6 samples
Epoch 1/30
14/14 [==============================] - 0s - loss: 8.1139 - acc: 0.4286 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 2/30
14/14 [==============================] - 0s - loss: 5.7565 - acc: 0.6429 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 3/30
14/14 [==============================] - 0s - loss: 5.8050 - acc: 0.6429 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 4/30
14/14 [==============================] - 0s - loss: 9.2603 - acc: 0.3571 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 5/30
14/14 [==============================] - 0s - loss: 8.0874 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 6/30
14/14 [==============================] - 0s - loss: 5.7565 - acc: 0.6429 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 7/30
14/14 [==============================] - 0s - loss: 5.7565 - acc: 0.6429 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 8/30
14/14 [==============================] - 0s - loss: 4.6052 - acc: 0.7143 - val_loss: 8.0590 - val_a

In [63]:
model_top.save_weights(path+'/Test_bottleneck_30_epochs.h5')

In [65]:
model_2 = applications.VGG16(weights='imagenet', include_top=False, input_shape=(3,150, 150))

In [71]:
model_2.output_shape

(None, 512, 4, 4)

In [72]:
top_model = Sequential()
top_model.add(Flatten(input_shape=model_2.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(len(nameOfClasses), activation='softmax'))
top_model.load_weights(path+'/Test_bottleneck_30_epochs.h5')

In [73]:
model = Model(inputs = model_2.input, outputs = top_model(model_2.output))

In [74]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 3, 150, 150)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 150, 150)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 150, 150)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 75, 75)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 75, 75)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 75, 75)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 128, 37, 37)       0         
__________

In [75]:
for layer in model.layers[:15]:
    layer.trainable = False

In [76]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [77]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size
        )

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size
        )

Found 14 images belonging to 2 classes.
Found 6 images belonging to 2 classes.


In [79]:
model.fit_generator(
    train_generator,
    steps_per_epoch=train_samples // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_samples // batch_size)

Epoch 1/10
14/14 [==============================] - 0s - loss: 5.8559 - acc: 0.5714 - val_loss: 5.3727 - val_acc: 0.6667
Epoch 2/10
14/14 [==============================] - 0s - loss: 8.1969 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 3/10
14/14 [==============================] - 0s - loss: 3.7317 - acc: 0.7143 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 4/10
14/14 [==============================] - 0s - loss: 4.8862 - acc: 0.5714 - val_loss: 5.3727 - val_acc: 0.6667
Epoch 5/10
14/14 [==============================] - 0s - loss: 5.9310 - acc: 0.5714 - val_loss: 5.3727 - val_acc: 0.6667
Epoch 6/10
14/14 [==============================] - 0s - loss: 5.9202 - acc: 0.5714 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 7/10
14/14 [==============================] - 0s - loss: 10.4355 - acc: 0.2857 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 8/10
14/14 [==============================] - 0s - loss: 4.7047 - acc: 0.6429 - val_loss: 10.7454 - val_acc: 0.3333
Epoch 9/10
14/14 [============

In [80]:
model.save_weights(path+'/FinalStaticModel_30epochs_vgg.h5')

In [81]:
import glob
import os
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [ ]:
img = load_img(,False, (img_width, img_height))
x = img_to_array(img)
predictions = model.predict(x.reshape((1,img_width, img_height,3)))
result = dict()
i = 0
for x in predictions.tolist()[0]:
        result[nameOfClasses[i]] = round(x,4)
        i+=1
print(result)

In [ ]:
import glob
import os
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
cwd = os.getcwd()
fname = glob.glob("/home/ubuntu/modeldata/data/validation/volcanic eruption/////*.jpg")
for i in range(len(fname)):    
    img = load_img(fname[i],False, (img_width, img_height))
    x = img_to_array(img)
    prediction = model.predict(x.reshape((1,img_width, img_height,3)),batch_size=32, verbose=0)          
    result = dict()
    i = 0
    for x in prediction.tolist()[0]:
            result[nameOfClasses[i]] = round(x,4)
            i+=1
    print(result)